In [2]:
import pandas as pd

In [3]:
questions = pd.read_csv("questions.csv")
paragraphs = pd.read_csv("paragraphs.csv")
sample = pd.read_csv("sample.csv")
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
# Test data
# It pairs all test question ids with all available paragraphs.
# 
# A question-paragraph id is generated by concatenating the id of the question to the symbol “#”
# and then to the paragraph id.
# 
# You will need to parse it to extract the two identifiers (e.g. question-id and paragraph-id). 
# 
# Format:
# id – identifier for question-paragraph pair 
# qpid – (question-id # para-id) – combined id or question and paragraph

test.head()

,id,qpid
0,0,1235#0
1,1,1235#1
2,2,1235#2
3,3,1235#3
4,4,1235#4


In [6]:
# This is your training data, showing the paragraphs that are relevant for the particular question.
# 
# Format:
# qid – id of question (from questions.csv)
# ParaId – paragraph id for correct answer (from paragraphs.csv)

train.head()

,qid,ParaId
0,580,21
1,1188,323
2,1072,226
3,15,499
4,589,28


In [7]:
# This is the what result of your model should look like and is the file that should be submitted for evaluation. 

# Format:
# id – identifier for question-paragraph pair (from test.csv)
# target (provided by your model) – 1 if para-id is in the top 5 relevant paragraphs for question q-id, 0 otherwise

sample.head()

,id,target
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [30]:
# One paragraph can be the answer to multiple questions

train.groupby('ParaId').count()

,qid
ParaId,
2,1
3,1
5,6
6,1
7,4
9,1
14,6
16,1
17,4
